In [ ]:
import pandas as pd
import os

In [ ]:
def make_flat_featureclass(variable_list, sample_level, year, field_order, geometry_fs, data_fs, fc_name, workspace):
    variables=pd.read_csv(variable_list)
    filtered_data = data_fs.loc[(data_fs['sample_level']==sample_level)& (data_fs['year_sample']==year)]
    filtered_data = filtered_data.loc[filtered_data['variable_code'].isin(variable_list['variable_code'])]
    grouped_data = categorize_values(filtered_data, variable_list, 'Broad Category', "")
    if sample_level == 'tract':
        index_fields = ['TRPAID', 'county', 'tract']
    else:
        index_fields = ['TRPAID', 'county', 'tract','block']
    flat_data = grouped_data.pivot(index = index_fields, columns='variable_name', values='value')
    flat_data = flat_data.reset_index()
    flat_data = flat_data[field_order]
    merged_df = pd.merge(geometry_fs, flat_data, on="TRPAID", how="inner")
    columns_drop = ['GlobalID', 'YEAR', 'created_date', 'created_user', 'last_edited_date', 'last_edited_user', 'Shape.STArea()', 'Shape.STLength()']
    merged_df = merged_df.drop(columns=columns_drop)
    merged_df.spatial.to_featureclass(os.path.join(workspace, "Tahoe_BlockGroup_2021_TDC_Values"), sanitize_columns=False)